# Actividad Guiada 3

### Extra Realizado

In [3]:
import urllib.request
import tsplib95
import math
import random
import gzip

file = "swiss42.tsp"

problem = tsplib95.load(file)

Nodes = list(problem.get_nodes())
Edges = list(problem.get_edges())

# prueba de librería

print("Costo de arista: "+str(problem.get_weight(0, 1)))


Costo de arista: 15


## Creación de Funciones


In [4]:
def cr_sol(Nodes):
    sol = [Nodes[0]]
    for n in Nodes[1:]:
        sol = sol+[random.choice(list(set(Nodes)-set({Nodes[0]})-set(sol)))]
    return sol


def distance(a, b, problem):
    return problem.get_weight(a, b)


def total_distance(sol, problem):
    total_distance = 0
    for i in range(len(sol)-1):
        total_distance += distance(sol[i], sol[i+1], problem)
    return total_distance+distance(sol[len(sol)-1], sol[0], problem)


## Búsqueda Aleatoria


In [5]:
def random_search(problem, N):
    nodes = list(problem.get_nodes())

    b_sol = []
    b_dis = float('inf')

    for i in range(N):
        sol = cr_sol(nodes)
        dis = total_distance(sol, problem)
        if dis < b_dis:
            b_sol = sol
            b_dis = dis

    print("Mejor Solución: ", b_sol)
    print("Mejor Distancia: ", b_dis)

    return b_sol


sol = random_search(problem, 10000)


Mejor Solución:  [0, 30, 37, 26, 21, 40, 27, 38, 41, 20, 15, 25, 12, 18, 4, 3, 32, 1, 17, 36, 35, 7, 8, 23, 22, 39, 9, 11, 29, 13, 19, 31, 16, 6, 14, 5, 2, 28, 10, 24, 33, 34]
Mejor Distancia:  3533


## Búsqueda Local


In [6]:
def gen_n(sol):

    b_sol = []
    b_dis = float('inf')

    for i in range(1, len(sol)-1):
        for j in range(i+1, len(sol)):
            s_nr = sol[:i]+[sol[j]]+sol[i+1:j]+[sol[i]]+sol[j+1:]

            dis_nr = total_distance(s_nr, problem)

            if dis_nr < b_dis:
                b_dis = dis_nr
                b_sol = s_nr
    return b_sol


print("Distancia Solucion Incial:", total_distance(sol, problem))

n_sol = gen_n(sol)

print("Distancia Mejor Solucion Local: ", total_distance(n_sol, problem))


Distancia Solucion Incial: 3533
Distancia Mejor Solucion Local:  3244


In [7]:
# local search

def local_search(problem):

    b_sol = []
    ref_sol = cr_sol(Nodes)
    b_dis = total_distance(ref_sol, problem)

    it = 0

    while (1):

        it += 1

        nr = gen_n(ref_sol)

        nr_dis = total_distance(nr, problem)

        if nr_dis < b_dis:
            b_sol = nr
            b_dis = nr_dis
        else:
            print("En la iteracion ", it,
                  ", la mejor solución encontrada es:", b_sol)
            print("Distancia     :", b_dis)
            return b_sol

        ref_sol = nr


sol = local_search(problem)


En la iteracion  35 , la mejor solución encontrada es: [0, 23, 40, 24, 38, 22, 39, 21, 9, 2, 27, 3, 4, 8, 41, 10, 25, 11, 12, 13, 19, 14, 16, 15, 37, 7, 1, 32, 30, 29, 28, 6, 18, 26, 5, 17, 31, 36, 35, 33, 34, 20]
Distancia     : 1794


## Mejora de alogoritmo:
Complejidad del Algoritmo: $n^{2}$

Mejora: Multiarranque con busqueda de mejores soluciones aleatorias.


In [8]:
import time  #libreria para detener nuestro algoritmo en función del tiempo.

def better_local_search(problem):

    b_sol = []
    ref_sol = cr_sol(Nodes)
    b_dis = total_distance(ref_sol, problem)
    b_dis_0=b_dis #variable para guardar la distancia aleatoria inicial.
    print("Distancia inicial: ",b_dis_0) # se imprime la misma

    sol_sets = [] 
    dis_sets = []

    it = 0
    it_2=0

    a = True # variable booleana que permitira ingresar en un proceso de multiarranque.
    rn=0
    #Tiempos en los cuales se detendran los algoritmos:
    timeout=time.time()+60*1.5 
    timeout_2=time.time()+5


    while (1):

        it += 1

        nr = gen_n(ref_sol)

        nr_dis = total_distance(nr, problem)

        if nr_dis < b_dis:
            b_sol = nr
            b_dis = nr_dis
        else:
            sol_sets.append(b_sol) # variables para guardar cada solucion con cada nuevo arranque iniciado
            dis_sets.append(b_dis)
            a = False  #se cambia el valor de la variable para ingresar en el proceso de multiarranque.

        if (it > 1000 or time.time()>timeout): #condición para detener el algoritmo despues de minuto y medio
            m=min(dis_sets) # obtención de la mínima distancia del grupo de distancias obtenidas.
            ## IMPRESIÓN DE RESULTADOS:
            print("Conjuntos de Distancias: ",dis_sets)
            print("Mejor Distancia: ", m)
            print("Mejor Solucion: ", sol_sets[dis_sets.index(m)])
            print("Número de soluciones aleatorias nuevas: ",rn)
            return b_sol

        if a == True:
            ref_sol = nr  # se ejecuta mientras sea posible encontrar una solucion mejor en la busqueda local.
        else:
           
            while (1):
              ref_sol = cr_sol(Nodes)
              b_dis = total_distance(ref_sol, problem)
              it_2+=1
              if b_dis<b_dis_0: #condición para encontrar una mejor solucion aleatoria entre las soluciones aleatorias encontradas anteriormente
                  b_dis_0=b_dis
                  break
              if time.time()>timeout_2:
                  timeout_2=time.time()+5 #condición de parada por tiempo cuando ya no se encuentran mejores soluciones aleatorias (5 segundos)
                  break

            rn+=1
            a = True


sol = better_local_search(problem)


Distancia inicial:  5119
Conjuntos de Distancias:  [1856, 1929, 1922, 1997, 1794, 1849, 1856, 1659, 1548, 1669, 1744, 1900, 1487, 1671, 2172, 1901, 1618, 1900, 1624, 1818, 1740, 1830, 1644, 1884]
Mejor Distancia:  1487
Mejor Solucion:  [0, 27, 2, 28, 30, 38, 22, 24, 40, 21, 39, 9, 23, 41, 8, 29, 34, 33, 20, 32, 1, 6, 7, 31, 35, 36, 17, 37, 15, 16, 14, 19, 13, 5, 26, 18, 10, 25, 11, 12, 4, 3]
Número de soluciones aleatorias nuevas:  24
